In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

circle_center_tangent_to_lines (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

In [3]:
add_coordinate!(robot, FrameOrigin("rh_ffdistal"); id="rh_ffdistal")
add_coordinate!(robot, FrameOrigin("rh_mfdistal"); id="rh_mfdistal")
add_coordinate!(robot, FrameOrigin("rh_rfdistal"); id="rh_rfdistal")
add_coordinate!(robot, FrameOrigin("rh_lfdistal"); id="rh_lfdistal")
add_coordinate!(robot, FrameOrigin("rh_thdistal"); id="rh_thdistal")
add_coordinate!(robot, FrameOrigin("rh_ffproximal"); id="rh_ffproximal")
add_coordinate!(robot, FrameOrigin("rh_thmiddle"); id="rh_thmiddle")

"rh_thmiddle"

### Hand Visualization

In [ ]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_ffmiddle_frame_id = get_compiled_frameID(m, "rh_fftip")
fftip_transform = get_transform(kcache, rh_fftip_frame_id)
display(fftip_transform.origin)

LoadError: UndefVarError: `rh_fftip_frame_id` not defined

In [ ]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.033, -0.01, 0.38]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.033
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.38
 0.0  0.0  0.0   1.0

In [ ]:
# Compile robot model
m = compile(robot)
kcache = new_kinematics_cache(m)  
medium_wrap_preshape = zeros(24)
medium_wrap_preshape[21] = 1.2 # thumb extended
kinematics!(kcache, 0.0, medium_wrap_preshape)

cylinder_radius = 0.01
cylinder_length = 0.2

# Get the positions of the finger tips
rh_fftip_frame_id = get_compiled_frameID(m, "rh_fftip")
fftip_transform = get_transform(kcache, rh_fftip_frame_id)
p11 = fftip_transform.origin

rh_ffmiddle_frame_id = get_compiled_frameID(m, "rh_ffmiddle")
ffmiddle_transform = get_transform(kcache, rh_ffmiddle_frame_id)
p12 = ffmiddle_transform.origin

rh_thtip_frame_id = get_compiled_frameID(m, "rh_thtip")
thtip_transform = get_transform(kcache, rh_thtip_frame_id)
p21 = thtip_transform.origin

rh_thmiddle_frame_id = get_compiled_frameID(m, "rh_thmiddle")
thmiddle_transform = get_transform(kcache, rh_thmiddle_frame_id)
p22 = thmiddle_transform.origin

if cylinder_radius < 0.015
    cylinder_position = SVector(0.0, -0.03, 0.32)
else
    cylinder_position = circle_center_tangent_to_lines(p11, p12, p21, p22, cylinder_radius + 0.01) # add one centimeter to the radius to avoid intersection with the fingers
    cylinder_position = SVector(cylinder_position[1], cylinder_position[2], cylinder_position[3])  # Convert to SVector
end

# Create the figure
fig = Figure(size=(800, 600))


# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

X = SVector(1., 0., 0.)
cylinder = Cylinder(Point3f(cylinder_position + (cylinder_length/2)*X), Point3f(cylinder_position - (cylinder_length/2)*X), cylinder_radius)
mesh!(ls, cylinder; color=:magenta, transparency=true)

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_thtip")
display_frame(m, ls, "rh_thdistal")
display_frame(m, ls, "world")
#display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [ ]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), frames(robot))

33-element Vector{String}:
 "world"
 "rh_forearm"
 "rh_wrist"
 "rh_palm"
 "rh_manipulator"
 "rh_imu"
 "rh_ffknuckle"
 "rh_ffproximal"
 "rh_ffmiddle"
 "rh_ffdistal"
 "rh_fftip"
 "rh_mfknuckle"
 "rh_mfproximal"
 "rh_mfmiddle"
 "rh_mfdistal"
 "rh_mftip"
 "rh_rfknuckle"
 "rh_rfproximal"
 "rh_rfmiddle"
 "rh_rfdistal"
 "rh_rftip"
 "rh_lfmetacarpal"
 "rh_lfknuckle"
 "rh_lfproximal"
 "rh_lfmiddle"
 "rh_lfdistal"
 "rh_lftip"
 "rh_thbase"
 "rh_thproximal"
 "rh_thhub"
 "rh_thmiddle"
 "rh_thdistal"
 "rh_thtip"

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [4]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 0.01, (limits.lower+0.0, limits.upper-0.0), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.0001, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Creation of the cylinder prismatic joints

In [5]:
vm = Mechanism{Float64}("VirtualTracks")

m = compile(robot)
kcache = new_kinematics_cache(m)  
medium_wrap_preshape = zeros(24)
medium_wrap_preshape[21] = 1.2 # thumb extended
kinematics!(kcache, 0.0, medium_wrap_preshape)

cylinder_radius = 0.03
cylinder_length = 0.2

# Get the positions of the finger tips
rh_fftip_frame_id = get_compiled_frameID(m, "rh_fftip")
fftip_transform = get_transform(kcache, rh_fftip_frame_id)
p11 = [fftip_transform.origin[2], fftip_transform.origin[3]]  

rh_ffmiddle_frame_id = get_compiled_frameID(m, "rh_ffmiddle")
ffmiddle_transform = get_transform(kcache, rh_ffmiddle_frame_id)
p12 = [ffmiddle_transform.origin[2], ffmiddle_transform.origin[3]]

rh_thtip_frame_id = get_compiled_frameID(m, "rh_thtip")
thtip_transform = get_transform(kcache, rh_thtip_frame_id)
p21 = [thtip_transform.origin[2], thtip_transform.origin[3]]

rh_thmiddle_frame_id = get_compiled_frameID(m, "rh_thmiddle")
thmiddle_transform = get_transform(kcache, rh_thmiddle_frame_id)
p22 = [thmiddle_transform.origin[2], thmiddle_transform.origin[3]]

if cylinder_radius < 0.015
    # add one centimeter to the radius to avoid intersection with the fingers 
    rh_ffknuckle_frame_id = get_compiled_frameID(m, "rh_ffknuckle")
    ffknuckle_transform = get_transform(kcache, rh_ffknuckle_frame_id)

    cylinder_position = SVector(0.0, -0.03, ffknuckle_transform.origin[3] - cylinder_radius - 0.007)
else
    # add one centimeter to the radius to avoid intersection with the fingers
    cylinder_position = circle_center_tangent_to_lines(p11, p12, p21, p22, cylinder_radius + 0.01) # add one centimeter to the radius to avoid intersection with the fingers
    cylinder_position = SVector(0.0, cylinder_position[1], cylinder_position[2])  # Convert to SVector
end

attracted_frames = ("rh_lfdistal_mass_coord", "rh_lfmiddle_mass_coord", "rh_lfproximal_mass_coord", "rh_rfdistal_mass_coord", 
                    "rh_rfmiddle_mass_coord", "rh_rfproximal_mass_coord", "rh_mfdistal_mass_coord", "rh_mfmiddle_mass_coord",
                    "rh_mfproximal_mass_coord", "rh_ffdistal_mass_coord", "rh_ffmiddle_mass_coord", "rh_ffproximal_mass_coord",
                    "rh_thdistal_mass_coord", "rh_thmiddle_mass_coord") #, "rh_thproximal_mass_coord" , "rh_palm_mass_coord")

attracted_frames_names = ("lfdistal", "lfmiddle", "lfprox", "rfdistal", "rfmiddle", "rfprox", "mfdistal", "mfmiddle", "mfprox", "ffdistal", "ffmiddle", 
                "ffprox", "thdistal", "thmiddle") #, "thprox", "palm")


for i in 1:length(attracted_frames)
    add_frame!(vm; id="center_frame_$(attracted_frames_names[i])")
    frame_pos = configuration(kcache, get_compiled_coordID(kcache, attracted_frames[i]))
    add_joint!(vm, Rigid(Transform(SVector(frame_pos[1], cylinder_position[2], cylinder_position[3]))); parent=root_frame(vm), child="center_frame_$(attracted_frames_names[i])", id="root_joint_$(attracted_frames_names[i])")
    add_frame!(vm; id="prism_frame_$(attracted_frames_names[i])")
    add_joint!(vm, Prismatic(SVector(1.0,0.0,0.0)); parent="center_frame_$(attracted_frames_names[i])", child="prism_frame_$(attracted_frames_names[i])", id="prism_joint_$(attracted_frames_names[i])")
    add_frame!(vm; id="revo_frame_$(attracted_frames_names[i])")
    add_joint!(vm, Revolute(SVector(1.0,0.0,0.0)); parent="prism_frame_$(attracted_frames_names[i])", child="revo_frame_$(attracted_frames_names[i])", id = "revo_joint_$(attracted_frames_names[i])")
    add_frame!(vm; id="ee_frame_$(attracted_frames_names[i])")
    add_joint!(vm, Rigid(Transform(SVector(0.0,0.0,cylinder_radius))); parent ="revo_frame_$(attracted_frames_names[i])", child ="ee_frame_$(attracted_frames_names[i])", id = "fixed_joint_$(attracted_frames_names[i])")

    add_coordinate!(vm, FrameOrigin("ee_frame_$(attracted_frames_names[i])"); id="$(attracted_frames_names[i]) ee position")
    add_component!(vm, PointMass(0.01, "$(attracted_frames_names[i]) ee position"); id="$(attracted_frames_names[i]) ee mass")

    add_coordinate!(vm, JointSubspace("prism_joint_$(attracted_frames_names[i])"); id="prism_joint_$(attracted_frames_names[i])")
    add_component!(vm, LinearDamper(0.1, "prism_joint_$(attracted_frames_names[i])"); id="prism_joint_$(attracted_frames_names[i])_damper")
    add_coordinate!(vm, JointSubspace("revo_joint_$(attracted_frames_names[i])"); id="revo_joint_$(attracted_frames_names[i])")
    #add_component!(vm, LinearDamper(0.005, "revo_joint_$(attracted_frames_names[i])"); id="revo_joint_$(attracted_frames_names[i])_damper")    

    add_coordinate!(vm, FrameOrigin("center_frame_$(attracted_frames_names[i])"); id="center_frame_$(attracted_frames_names[i])")
    add_coordinate!(vm, FrameOrigin("prism_frame_$(attracted_frames_names[i])"); id="prism_frame_$(attracted_frames_names[i])")
    add_coordinate!(vm, CoordDifference("center_frame_$(attracted_frames_names[i])", "prism_frame_$(attracted_frames_names[i])"); id="$(attracted_frames_names[i])_prismatic_error")
    comeback_stiffness = 0.1
    comeback_stiffness_matrix = SMatrix{3, 3}(comeback_stiffness, 0., 0., 0., comeback_stiffness, 0., 0., 0., comeback_stiffness)
    add_component!(vm, LinearSpring(comeback_stiffness_matrix, "$(attracted_frames_names[i])_prismatic_error"); id = "$(attracted_frames_names[i])_comeback_spring")
end

# WHY NOT REDUCING THE LAST RIGID JOINT (FOR THE LITTLE FINGER) SUCH THAT THIS FINGER EXERTS MORE FORCE? (read in a paper that this finger exerts more force) 
# ---> should also adapt the collision model then

add_gravity_compensation!(vm, VMRobotControl.DEFAULT_GRAVITY)

### Addition of the multiples springs/dampers

Hand motion

In [ ]:
vms = VirtualMechanismSystem("myShadowVMS", robot, vm)
root = root_frame(vms.robot)

D = SMatrix{3, 3}(0.05, 0., 0., 0., 0.05, 0., 0., 0., 0.05)

base_stiffness = 0.05

phalanx_scaling_factor = 0.5  #phalanx scaling : distal stiffness bigger than proximal
finger_scaling_factor = 1.5 #finger scaling : thumb stiffness bigger than little finger

stiffnesses = generate_stiffnesses_geometric_scaling(base_stiffness, phalanx_scaling_factor, finger_scaling_factor)

damping_decay_rate = 161 # 20% of damping at |z| = 0.01
#damping_decay_rate = 460 # 20% of damping at |z| = 0.005
exponential_damping_coeff = 0.1

for i in 1:length(attracted_frames)
    K = SMatrix{3, 3}(stiffnesses[i], 0., 0., 0., stiffnesses[i], 0., 0., 0., stiffnesses[i])
    add_coordinate!(vms, CoordDifference(".virtual_mechanism.$(attracted_frames_names[i]) ee position", ".robot.$(attracted_frames[i])"); id = "ee $(attracted_frames_names[i]) diff")
    add_component!(vms, LinearSpring(K, "ee $(attracted_frames_names[i]) diff"); id = "ee $(attracted_frames_names[i]) spring")
    add_component!(vms, LinearDamper(D, "ee $(attracted_frames_names[i]) diff"); id = "ee $(attracted_frames_names[i]) damper")

    exponential_damping_matrix = SMatrix{3, 3}(exponential_damping_coeff, 0., 0., 0., exponential_damping_coeff, 0., 0., 0., exponential_damping_coeff)
    add_component!(vms, ExponentialDamper(exponential_damping_matrix, "ee $(attracted_frames_names[i]) diff", damping_decay_rate); id = "ee $(attracted_frames_names[i]) exp damper")
end

thumb_massive_damping = 10.0
add_component!(vms, LinearDamper(SMatrix{3, 3}(thumb_massive_damping, 0., 0., 0., thumb_massive_damping, 0., 0., 0., thumb_massive_damping), "ee thmiddle diff"); id = "thmiddle mass damper")

#lightly constraint some joints to avoid unwanted motions 
add_component!(vms, LinearSpring(0.01, ".robot.rh_FFJ4_coord"); id = "ff j4 angular spring")
add_component!(vms, LinearSpring(0.01, ".robot.rh_MFJ4_coord"); id = "mf j4 angular spring")
add_component!(vms, LinearSpring(0.01, ".robot.rh_RFJ4_coord"); id = "rf j4 angular spring")
add_component!(vms, LinearSpring(0.01, ".robot.rh_LFJ4_coord"); id = "lf j4 angular spring")
add_component!(vms, LinearSpring(0.01, ".robot.rh_WRJ1_coord"); id = "wr j1 angular spring")
add_component!(vms, LinearSpring(0.01, ".robot.rh_WRJ2_coord"); id = "wr j2 angular spring")


"wr j2 angular spring"

Cylinder collision model

In [149]:
add_coordinate!(vms,  ConstCoord(cylinder_position);  id="cylinder position")
add_coordinate!(vms, ConstCoord(cylinder_radius); id="cylinder radius")

add_coordinate!(vms, FramePoint(".robot.rh_palm", SVector(0. , 0., 0.07)); id="second palm point")

repulsed_frames = (".robot.rh_fftip_mass_coord", ".robot.rh_mftip_mass_coord", ".robot.rh_rftip_mass_coord",".robot.rh_lftip_mass_coord" , 
                    ".robot.rh_thtip_mass_coord", ".robot.rh_ffmiddle_mass_coord",".robot.rh_mfmiddle_mass_coord", ".robot.rh_rfmiddle_mass_coord",
                    ".robot.rh_lfmiddle_mass_coord",  ".robot.rh_thmiddle_mass_coord", ".robot.rh_ffproximal_mass_coord", ".robot.rh_mfproximal_mass_coord",
                    ".robot.rh_rfproximal_mass_coord", ".robot.rh_lfproximal_mass_coord", ".robot.rh_thproximal_mass_coord", ".robot.rh_palm_mass_coord", "second palm point",
                    ".robot.rh_ffdistal", ".robot.rh_mfdistal", ".robot.rh_rfdistal", ".robot.rh_lfdistal", ".robot.rh_thdistal", ".robot.rh_thmiddle")
repulsed_frames_names = ("fftip", "mftip", "rftip", "lftip", "thtip", "ffmiddle", "mfmiddle", "rfmiddle", "lfmiddle", "thmiddle", "ffprox", 
                "mfprox", "rfprox", "lfprox", "thprox", "palm", "palm2", "ffdistal", "mfdistal", "rfdistal", "lfdistal", "thdistal", "thmiddle2")

for i in 1:length(repulsed_frames)
    frame = repulsed_frames[i]
    add_coordinate!(vms, CoordDifference(frame, "cylinder position") ; id = "$(repulsed_frames_names[i]) cylinder diff" )
    add_coordinate!(vms, CoordSlice("$(repulsed_frames_names[i]) cylinder diff", SVector(2,3)); id="$(repulsed_frames_names[i]) planar error")
    add_coordinate!(vms, CoordNorm("$(repulsed_frames_names[i]) planar error") ; id = "$(repulsed_frames_names[i]) planar error norm")
    add_coordinate!(vms, CoordDifference("$(repulsed_frames_names[i]) planar error norm", "cylinder radius"); id = "shifted $(repulsed_frames_names[i]) cylinder error" )

    add_component!(vms, ReLUSpring(5.0, "shifted $(repulsed_frames_names[i]) cylinder error", true); id="$(repulsed_frames_names[i]) cylinder repulsive spring")
    add_component!(vms, RectifiedDamper(5.0, "$(repulsed_frames_names[i]) planar error norm", (0.0, 1.1*cylinder_radius), true, false); id="$(repulsed_frames_names[i]) cylinder damper")
end

## Simulating the Robot

### Setting Up the Simulation

In [150]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())


tspan = (0., 15.)
vms_compiled = compile(vms)

q_init = zeros(24)
q_init[21] = 1.2
q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan)
@info "Simulating shadow robot with cylinder object centric motion"

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-3, reltol=1e-3);

[ Info: Simulating shadow robot with cylinder object centric motion
ODE   0%|█                                              |  ETA: N/A
ODE 100%|███████████████████████████████████████████████| Time: 0:00:04


### Visualizing the Results

In [151]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.6]
#cam.eyeposition[] = [0.25, -0.03, 0.29]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
robotsketch!(ls, plotting_vm_kcache; scale = 0.05)

X = SVector(1., 0., 0.)
cylinder = Cylinder(Point3f(cylinder_position + (cylinder_length/2)*X), Point3f(cylinder_position - (cylinder_length/2)*X), cylinder_radius)
mesh!(ls, cylinder; color=:magenta, transparency=true)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_object_centric_cylinder_2.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath;fps=20,  t=plotting_t);

LoadError: Screen not open!